# Recommendation System

In [1]:
from collections import Counter
from itertools import combinations
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
import seaborn as sns

import pickle as pk

from jupyterthemes import jtplot
jtplot.style(figsize=(15, 9))

## Data

In [3]:
movies = pd.read_csv('data/movies_dataset_CLEAN_noNaN.csv', index_col='id')

# Vectors
vectors_keywords = pd.read_csv('data/keywords_vectors.csv', index_col=0)
vectors_cast     = pd.read_csv('data/cast_vectors.csv', index_col=0)
vectors_crew     = pd.read_csv('data/crew_vectors.csv', index_col=0)

# Sum of Vectors
sum_vectors_keywords = pd.read_csv('data/keywords_sum_vectors.csv', index_col='id')
sum_vectors_cast     = pd.read_csv('data/cast_sum_vectors.csv', index_col='id')
sum_vectors_crew     = pd.read_csv('data/crew_sum_vectors.csv', index_col='id')

## Movies choice

In [29]:
like = [
    'From Dusk Till Dawn',
    'Alien',
    'The Descent',
]

In [22]:
def i_want_to_watch_something_like(like:list)->pd.DataFrame:
    return movies[movies['title'].isin(like)]

In [23]:
i_want_to_watch_something_like(like)

,genre_ids,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords,label,label_encoder
id,,,,,,,,,,,,,
348,"[27, 878]","During its return to the earth, commercial spa...",62.215,1979-05-25,Alien,8.1,12914,"Horror, Science Fiction","[{'cast_id': 10205, 'name': 'Sigourney Weaver'...","[{'crew_id': 578, 'name': 'Ridley Scott', 'dep...","[{'id': 803, 'name': 'android'}, {'id': 1612, ...",Science Fiction,14
755,"[27, 28, 53, 80]",Seth Gecko and his younger brother Richard are...,27.675,1996-01-19,From Dusk Till Dawn,7.0,5301,"Horror, Action, Thriller, Crime","[{'cast_id': 1461, 'name': 'George Clooney', '...","[{'crew_id': 2294, 'name': 'Robert Rodriguez',...","[{'id': 246, 'name': 'dancing'}, {'id': 380, '...",Horror,10
9392,"[12, 27]","After a tragic accident, six friends reunite f...",22.417,2005-07-08,The Descent,7.0,3394,"Adventure, Horror","[{'cast_id': 57574, 'name': 'Shauna Macdonald'...","[{'crew_id': 558, 'name': 'David Julyan', 'dep...","[{'id': 1852, 'name': 'mutant'}, {'id': 1963, ...",Horror,10


In [27]:
class IWantToWatchSomethingLike:
    def __init__(self, like:list):
        self.like = like
        
    def show_what_i_like(self)->pd.DataFrame:
        return movies[movies['title'].isin(self.like)]

In [28]:
what_to_watch = IWantToWatchSomethingLike(like)

what_to_watch.show_what_i_like()

,genre_ids,overview,popularity,release_date,title,vote_average,vote_count,genres,cast,crew,keywords,label,label_encoder
id,,,,,,,,,,,,,
348,"[27, 878]","During its return to the earth, commercial spa...",62.215,1979-05-25,Alien,8.1,12914,"Horror, Science Fiction","[{'cast_id': 10205, 'name': 'Sigourney Weaver'...","[{'crew_id': 578, 'name': 'Ridley Scott', 'dep...","[{'id': 803, 'name': 'android'}, {'id': 1612, ...",Science Fiction,14
755,"[27, 28, 53, 80]",Seth Gecko and his younger brother Richard are...,27.675,1996-01-19,From Dusk Till Dawn,7.0,5301,"Horror, Action, Thriller, Crime","[{'cast_id': 1461, 'name': 'George Clooney', '...","[{'crew_id': 2294, 'name': 'Robert Rodriguez',...","[{'id': 246, 'name': 'dancing'}, {'id': 380, '...",Horror,10
9392,"[12, 27]","After a tragic accident, six friends reunite f...",22.417,2005-07-08,The Descent,7.0,3394,"Adventure, Horror","[{'cast_id': 57574, 'name': 'Shauna Macdonald'...","[{'crew_id': 558, 'name': 'David Julyan', 'dep...","[{'id': 1852, 'name': 'mutant'}, {'id': 1963, ...",Horror,10
